# Roadmap Detalhado para Implementação do Multi-GNN

**Análise**: A pesquisa no repositório `IBM/multi-gnn` e a análise dos scripts `test_benchmark_improved.py` e `test_benchmark_integration.py` revelam que o processo de benchmark já foi parcialmente explorado. O principal desafio é a complexidade do ambiente (PyTorch Geometric no Windows) e a necessidade de adaptar o código original para gerar artefatos de predição, em vez de apenas métricas de F1-Score.

Este notebook serve como um guia passo a passo para implementar, treinar e extrair predições do modelo Multi-GNN.

---

## Fase 0: Ambiente e Pré-requisitos

**Problema**: O Multi-GNN depende de bibliotecas (torch-scatter, torch-sparse) que são notoriamente difíceis de instalar e usar diretamente no Windows. O script `test_benchmark_improved.py` corretamente identifica isso.

**Solução**: Usar o **Windows Subsystem for Linux (WSL2)**. Ele permite executar um ambiente Linux diretamente no Windows, eliminando todos os problemas de compatibilidade.

### Ações:
1.  **Instale o WSL2**: Se ainda não o tiver, abra o PowerShell como administrador e execute:
    ```bash
    wsl --install
    ```
2.  **Clone o Repositório no WSL2**: Navegue até o diretório do seu projeto dentro do WSL2 (ex: `/mnt/c/Users/gafeb/AML_project`) e execute os comandos a partir daí.
3.  **Crie o Ambiente Conda**: O repositório `benchmarks/Multi-GNN` já deve conter o arquivo `env.yml`. Crie o ambiente com o comando:
    ```bash
    conda env create -f benchmarks/Multi-GNN/env.yml
    ```
4.  **Ative o Ambiente**:
    ```bash
    conda activate multignn
    ```

## Fase 1: Preparação dos Dados

O Multi-GNN requer um formato de dados específico. O script `format_kaggle_files.py` é responsável por essa conversão.

### Ações:
1.  **Baixe os Dados**: Faça o download dos dados do [Kaggle](https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml/data) e coloque o arquivo `HI-Small_Trans.csv` na pasta `data/raw/`.
2.  **Execute o Script de Formatação**: Dentro do ambiente `multignn` no WSL2, execute:
    ```bash
    python benchmarks/Multi-GNN/format_kaggle_files.py data/raw/HI-Small_Trans.csv
    ```
    Isso criará um arquivo `formatted_transactions.csv` no mesmo diretório.
3.  **Configure o `data_config.json`**: Este arquivo é crucial. Ele informa ao `main.py` onde encontrar os dados.
    -   **Mova** o `formatted_transactions.csv` para uma pasta dedicada, por exemplo: `data/processed/multi-gnn/Small_HI/`.
    -   **Abra** o arquivo `benchmarks/Multi-GNN/data_config.json`.
    -   **Atualize** o caminho em `aml_data` para apontar para o diretório que você criou (ex: `"aml_data": "data/processed/multi-gnn/"`).

---

## Fase 2: Estrutura do Grafo (Análise)

Não há ação de codificação aqui, mas é importante entender como o grafo é construído. O script `data_loading.py` do repositório da IBM:
-   Lê o `formatted_transactions.csv`.
-   Cria **nós** para cada conta (`from_id`, `to_id`).
-   Cria **arestas** para cada transação.
-   Adiciona **features de aresta**, como `Timestamp` e `Amount`.

---

## Fase 3: Treinamento do Modelo

Com o ambiente e os dados prontos, podemos treinar o modelo.

### Ação:
1.  **Execute o Treinamento**: Use o comando abaixo no WSL2. Ele é derivado do `test_benchmark_improved.py`, mas simplificado.
    ```bash
    python benchmarks/Multi-GNN/main.py --data Small_HI --model gin --emlps --reverse_mp --testing --n_epochs 5
    ```
    -   `--data Small_HI`: Corresponde ao nome da pasta dos dados.
    -   `--model gin`: Usa o modelo GIN, um dos mais eficazes.
    -   `--emlps --reverse_mp`: Adaptações do modelo que melhoram a performance.
    -   `--testing`: **Importante**: Desabilita o logging no `wandb`, evitando a necessidade de configuração de API.
    -   `--n_epochs 5`: Apenas 5 épocas para um teste rápido. Aumente para 50-100 para um treinamento completo.

---

## Fase 4: Geração de Artefatos (Modificação Essencial)

**Problema**: O código original da IBM foi feito para pesquisa e apenas imprime o F1-Score. Ele **não salva as predições**, que é o que precisamos para o benchmark.

**Solução**: Faremos uma pequena modificação no código do Multi-GNN para salvar as probabilidades de predição (`preds`) e os identificadores verdadeiros (`ground_truth`) em um arquivo CSV.

### Ação:
1.  **Modifique o arquivo `train_util.py`**:
    -   Abra o arquivo: `benchmarks/Multi-GNN/train_util.py`.
    -   Encontre a função `evaluate_homo` (por volta da linha 99).
    -   **Adicione o seguinte código** no final da função, logo antes da linha `return f1`:

    ```python
    # --- INÍCIO DA MODIFICAÇÃO PARA SALVAR PREDIÇÕES ---
    import pandas as pd
    import os

    # Salvar apenas durante a avaliação final (não na validação)
    # Podemos checar o número de arestas no loader para diferenciar teste de validação.
    # Esta é uma heurística; uma solução mais robusta passaria um argumento extra.
    is_test_set = (loader.data.edge_attr.shape[0] == te_data.edge_attr.shape[0])

    if is_test_set and args.save_model: # Usamos o save_model como gatilho
        print("Salvando predições do conjunto de teste...")
        
        # Extrai os IDs originais das arestas (transações)
        # O loader embaralha os dados, então precisamos dos `input_id` para mapear de volta
        original_edge_ids = inds[batch.input_id.detach().cpu()].numpy()

        output_df = pd.DataFrame({
            'edge_id': original_edge_ids,
            'prediction_prob': out[:, 1].detach().cpu().numpy(), # Probabilidade da classe 1 (fraude)
            'ground_truth': batch.y[mask].cpu().numpy()
        })
        
        output_path = "multi_gnn_predictions.csv"
        output_df.to_csv(output_path, index=False)
        print(f"Predições salvas em {os.path.abspath(output_path)}")
    # --- FIM DA MODIFICAÇÃO ---

    return f1
    ```
    *Nota: A lógica acima é um exemplo. A implementação exata pode precisar de ajustes para capturar os tensores corretos (`out`, `batch.y`, etc.) dependendo da versão do PyG.*

---

## Fase 5: Execução Final e Integração

Agora, execute o treinamento novamente, mas desta vez adicione o flag `--save_model` para ativar a lógica de salvamento que acabamos de adicionar.

### Ações:
1.  **Execute o Treinamento Final**:
    ```bash
    python benchmarks/Multi-GNN/main.py --data Small_HI --model gin --emlps --reverse_mp --testing --n_epochs 50 --save_model --unique_name aml_benchmark
    ```
    -   `--save_model`: Ativa o salvamento do modelo e, crucialmente, o nosso código de salvamento de predições.
    -   `--unique_name aml_benchmark`: Nome para o checkpoint do modelo.

2.  **Verifique o Artefato**: Após a execução, um arquivo `multi_gnn_predictions.csv` deve aparecer na raiz do projeto.

3.  **Integre no Notebook Principal**: Agora você pode carregar este CSV no notebook `03_Modelagem_e_Avaliacao.ipynb` e comparar as probabilidades do Multi-GNN com as do seu modelo XGBoost no mesmo conjunto de teste.